In [6]:
# 导入必要的库，包括文件操作、模块导入、日期时间处理，以及项目中定义的超启发式算法类、LLM客户端工具和搜索文件工具
import os
import importlib
from datetime import datetime
from src.pipeline.hyper_heuristics.random import RandomHyperHeuristic
from src.pipeline.hyper_heuristics.single import SingleHyperHeuristic
from src.pipeline.hyper_heuristics.llm_selection import LLMSelectionHyperHeuristic
from src.util.llm_client.get_llm_client import get_llm_client
from src.util.util import search_file

# --------------------------
# 在 Notebook 中设置参数（替代命令行参数）

# --------------------------
# 定义问题类型为"tsp"（旅行商问题），可替换为其他问题如"mkp"等
problem = "psp"  # 替换为实际问题名称（如 tsp、mkp 等）
# 指定使用的启发式策略为"llm_hh"（基于LLM的启发式选择），可选值还包括具体启发式函数名、"random_hh"（随机选择）、"or_solver"（精确求解器）
heuristic = "greedy_eta_assignment_8222"  # 可选值：具体启发式函数名、llm_hh、random_hh、or_solver
# LLM配置文件路径，默认使用azure_gpt_4o.json
llm_config_file = os.path.join("output", "llm_config", "azure_gpt_4o.json")  # LLM配置文件路径
# 启发式函数所在目录，默认为"basic_heuristics"
heuristic_dir = "evovled_heuristics"  # 启发式函数目录
# 测试数据路径或目录，默认为"test_data"（表示使用问题对应的test_data目录下的所有文件）
test_data = "test_data"  # 测试数据路径或目录
# 迭代次数缩放因子，决定总启发式步骤与问题规模的比例，默认2.0
iterations_scale_factor = 4.0  # 迭代次数缩放因子
# LLM模式下每次选择启发式后执行的步数，默认5
steps_per_selection = 1  # LLM模式下每次选择的执行步数
# LLM模式下考虑的候选启发式数量，1表示不使用TTS直接选择，默认1
num_candidate_heuristics = 1  # LLM模式下考虑的候选启发式数量
# LLM模式下每个启发式的蒙特卡洛评估次数，0表示不使用TTS，默认0
rollout_budget = 10  # 蒙特卡洛评估次数
# 结果保存目录，默认"result"
result_dir = "result"  # 结果保存目录

# --------------------------
# 核心执行逻辑（无需修改）
# --------------------------
# 定义运行超启发式算法的函数，接收上述所有参数
def run_hyper_heuristic(
    problem,
    heuristic,
    llm_config_file,
    heuristic_dir,
    test_data,
    iterations_scale_factor,
    steps_per_selection,
    num_candidate_heuristics,
    rollout_budget,
    result_dir
):
    # 生成当前时间字符串，用于构建实验名称
    datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    # '20250916_102701'
    # 提取启发式名称（去除路径和扩展名）
    heuristic_name = heuristic.split(os.sep)[-1].split(".")[0]
    
    # 构建启发式池路径，并检查路径是否存在
    heuristic_pool_path = os.path.join("src", "problems", problem, "heuristics", heuristic_dir)
    if not os.path.exists(heuristic_pool_path):
        raise FileNotFoundError(f"启发式目录不存在: {heuristic_pool_path}")
    # 获取启发式池中的所有文件（即可用的启发式函数），排除.ipynb_checkpoints文件夹
    heuristic_pool = [
        f for f in os.listdir(heuristic_pool_path) 
        if f != '.ipynb_checkpoints'  # 排除checkpoints文件夹
    ]

    # 确定基础输出目录，优先使用环境变量AMLT_OUTPUT_DIR，否则默认"output"
    base_output_dir = os.path.join(os.getenv("AMLT_OUTPUT_DIR"), "..", "..", "output") if os.getenv("AMLT_OUTPUT_DIR") else "output"

    # 初始化超启发式实例、实验名称和LLM客户端
    hyper_heuristic = None
    experiment_name = ""
    llm_client = None

    # 如果使用"llm_hh"策略
    if heuristic_name == "llm_hh":
        # 定义提示词目录，加载LLM客户端
        prompt_dir = os.path.join("src", "problems", "base", "prompt")
        llm_client = get_llm_client(llm_config_file, prompt_dir, None)
        # 提取LLM名称（从配置文件名），构建实验名称
        llm_name = os.path.splitext(os.path.basename(llm_config_file))[0]
        experiment_name = f"{heuristic_name}.{heuristic_dir}.{llm_name}.n{iterations_scale_factor}m{steps_per_selection}c{num_candidate_heuristics}b{rollout_budget}.{datetime_str}"
        # 初始化LLM选择超启发式实例
        hyper_heuristic = LLMSelectionHyperHeuristic(
            llm_client=llm_client,
            heuristic_pool=heuristic_pool,
            problem=problem,
            iterations_scale_factor=iterations_scale_factor,
            steps_per_selection=steps_per_selection,
            num_candidate_heuristics=num_candidate_heuristics,
            rollout_budget=rollout_budget,
        )
    # 如果使用"random_hh"策略
    elif heuristic_name == "random_hh":
        # 构建实验名称，初始化随机选择超启发式实例
        experiment_name = f"{heuristic_name}.{heuristic_dir}.{datetime_str}"
        print('experiment_name===============',experiment_name)
        print(heuristic_pool)
        hyper_heuristic = RandomHyperHeuristic(
            heuristic_pool=heuristic_pool,
            problem=problem,
            iterations_scale_factor=iterations_scale_factor
        )

    # 如果使用"or_solver"策略
    elif heuristic_name == "or_solver":
        # 实验名称为"or_solver"，导入并初始化OR精确求解器
        experiment_name = "or_solver"
        module = importlib.import_module(f"src.problems.{problem}.or_solver")
        ORSolver = getattr(module, "ORSolver")
        hyper_heuristic = ORSolver(problem=problem)
    # 如果使用具体的启发式函数
    else:
        # 实验名称为启发式函数名，初始化单一启发式实例
        experiment_name = heuristic_name
        hyper_heuristic = SingleHyperHeuristic(heuristic=heuristic_name, problem=problem, iterations_scale_factor=iterations_scale_factor)

    # 导入当前问题对应的环境类Env
    try:
        module = importlib.import_module(f"src.problems.{problem}.env")
        Env = getattr(module, "Env")
    except Exception as e:
        raise ImportError(f"无法导入问题 {problem} 的环境类: {str(e)}")

    # 处理测试数据：如果test_data为"test_data"，则获取该目录下的所有文件；否则使用指定的单个文件
    # 处理测试数据（新增过滤逻辑）
    if test_data == "test_data":
        test_data_dir = search_file("test_data", problem)
        # 过滤掉 .ipynb_checkpoints 文件夹
        test_data_list = [
            f for f in os.listdir(test_data_dir) 
            if f != ".ipynb_checkpoints"  # 排除Jupyter自动生成的检查点文件夹
        ]
    else:
        test_data_list = [test_data]

    # 遍历每个测试数据，执行超启发式算法
    for data_name in test_data_list:
        try:
            # 初始化环境实例，设置输出目录并重置环境
            env = Env(data_name=data_name)
            output_dir = os.path.join(base_output_dir, problem, result_dir, env.data_ref_name, experiment_name)
            env.reset(output_dir)

            # 保存参数到parameters.txt文件
            params = {
                "problem": problem,
                "heuristic": heuristic,
                "llm_config_file": llm_config_file,
                "heuristic_dir": heuristic_dir,
                "test_data": test_data,
                "iterations_scale_factor": iterations_scale_factor,
                "steps_per_selection": steps_per_selection,
                "num_candidate_heuristics": num_candidate_heuristics,
                "rollout_budget": rollout_budget,
                "result_dir": result_dir,
                "data_path": env.data_path # 存储具体问题数据的文件所在的位置
            }
            with open(os.path.join(env.output_dir, "parameters.txt"), 'w') as f:
                f.write('\n'.join(f'{k}={v}' for k, v in params.items()))

            # 运行超启发式算法：如果是llm_hh，先重置LLM客户端的输出目录
            if heuristic_name == "llm_hh":
                llm_client.reset(env.output_dir)
            validation_result = hyper_heuristic.run(env)

            # 输出结果：如果验证成功，保存结果并打印成功信息；否则打印失败信息
            if validation_result:
                env.dump_result()
                print(f"成功: {os.path.join(env.output_dir, 'result.txt')} | {heuristic_name} | {data_name} | {env.key_item}: {env.key_value}")
            else:
                print(f"失败: 无效解 | {heuristic_name} | {data_name}")
        # 捕获并打印处理当前测试数据时的错误
        except Exception as e:
            print(f"处理 {data_name} 时出错: {str(e)}")

# --------------------------
# 执行（直接运行该单元格即可）
# --------------------------
# 调用run_hyper_heuristic函数，传入设置的参数，执行超启发式算法
run_hyper_heuristic(
    problem=problem,
    heuristic=heuristic,
    llm_config_file=llm_config_file,
    heuristic_dir=heuristic_dir,
    test_data=test_data,
    iterations_scale_factor=iterations_scale_factor,
    steps_per_selection=steps_per_selection,
    num_candidate_heuristics=num_candidate_heuristics,
    rollout_budget=rollout_budget,
    result_dir=result_dir
)

成功: output\psp\result\realistic_port_01_v6_b3_t8_T12.txt\greedy_eta_assignment_8222\result.txt | greedy_eta_assignment_8222 | realistic_port_01_v6_b3_t8_T12.txt | total_scheduling_cost: 12731.696
成功: output\psp\result\realistic_port_02_v8_b4_t10_T12.txt\greedy_eta_assignment_8222\result.txt | greedy_eta_assignment_8222 | realistic_port_02_v8_b4_t10_T12.txt | total_scheduling_cost: 21949.287999999997
成功: output\psp\result\realistic_port_03_v10_b5_t12_T12.txt\greedy_eta_assignment_8222\result.txt | greedy_eta_assignment_8222 | realistic_port_03_v10_b5_t12_T12.txt | total_scheduling_cost: 28288.468
成功: output\psp\result\realistic_port_04_v12_b5_t15_T24.txt\greedy_eta_assignment_8222\result.txt | greedy_eta_assignment_8222 | realistic_port_04_v12_b5_t15_T24.txt | total_scheduling_cost: 9061.716499999999
成功: output\psp\result\realistic_port_05_v15_b6_t18_T24.txt\greedy_eta_assignment_8222\result.txt | greedy_eta_assignment_8222 | realistic_port_05_v15_b6_t18_T24.txt | total_scheduling_cost: